In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

* https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html
* https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.stem.html

Demos:
* https://matplotlib.org/stable/gallery/mplot3d/

## Basic 3D plots

In [ ]:
filename = "../03-week3/catretina.csv"
data = np.genfromtxt(filename, delimiter=",", names=True, skip_header=1)
collumns = list(data.dtype.names)

print(collumns)

x = data["retinarea"]
y = data["age"]
z = data["cpRatio"]

In [ ]:
fig, axs = plt.subplots(ncols=3)
fig.tight_layout(pad=2.0)  # add some space

axs[0].plot(x, z, "x-")
axs[0].set_xlabel("retinarea")
axs[0].set_ylabel("cpRatio")

axs[1].plot(y, z, "x-")
axs[1].set_xlabel("age")
axs[1].set_ylabel("cpRatio")

axs[2].plot(y, x, "x-")
axs[2].set_xlabel("age")
axs[2].set_ylabel("retinarea")
plt.show()

In [ ]:
ax = plt.axes(projection="3d")
ax.scatter(x, y, z)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

plt.show()

In [ ]:
ax = plt.axes(projection="3d")

ax.set_box_aspect(aspect=None, zoom=0.85)

ax.stem(x, y, z)

ax.view_init(45.0, 60.0, 0.0)
ax.set_xlabel("retinarea (mm$^2$)")
ax.set_ylabel("age")
ax.set_zlabel("cpRatio")

plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# ax.stem(x, y, z)
ax.set(xlabel="x", ylabel="y", zlabel="z")

ax.stem(x, y, z, orientation="x")
ax.stem(x, y, z, orientation="z")
plt.show()

## Heatmaps

* http://www.bom.gov.au/climate/maps/averages/temperature/
* http://www.bom.gov.au/web01/ncc/www/climatology/temperature/mxt/mxtan.zip
* Data is 2D array
* data[i, j] is the temp. at x-position i, y-position j

In [ ]:
import seaborn as sns

maxann = np.genfromtxt("maxann.txt", skip_header=6, delimiter=" ")

In [ ]:
sns.heatmap(maxann)
plt.show()

Remove the "no data" flags: -9999

In [ ]:
maxann[maxann == -9999] = np.nan

ax = sns.heatmap(maxann)
ax.collections[0].colorbar.set_label("Maximum Annual Temperature (1991-2020)")
plt.show()

Attempt to label axis

In [ ]:
xmin = 112.0
nx = 1681

ymin = -44.0
ny = 1361

dx = 0.025

x = np.linspace(xmin, xmin + dx * (nx - 1), nx)
y = np.linspace(ymin, ymin + dx * (nx - 1), ny)


every = 100
ylabels = []
for i, ty in enumerate(y):
    if i % every == 0:
        ylabels.append(f"{ty:.0f}")
ylabels.reverse()

xlabels = []
for i, tx in enumerate(x):
    if i % every == 0:
        xlabels.append(f"{tx:.0f}")

ax = sns.heatmap(maxann)
ax.collections[0].colorbar.set_label("Maximum Annual Temperature (1991-2020)")
ax.set_yticks(range(0, ny, every))
ax.set_yticklabels(ylabels)
ax.set_xticks(range(0, nx, every))
ax.set_xticklabels(xlabels)
ax.set_xlabel("Longtitude")
ax.set_ylabel("Lattitude")
plt.show()

In [ ]:
from matplotlib import cm

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_box_aspect(aspect=None, zoom=0.85)

X, Y = np.meshgrid(x, y)

# Plot the surface
ax.plot_surface(X, Y, maxann, cmap=cm.coolwarm)

ax.view_init(-50, -70, 0)

ax.set_xlabel("Longtitude")
ax.set_ylabel("Lattitude")
ax.set_zlabel("Temp")

plt.show()

### Spatial Correlation

Zoom in on Tassie:

In [ ]:
tassie = maxann[1210:, 1281:1464]
ax = sns.heatmap(tassie)

In [ ]:
t0 = np.nanmean(tassie)

print(t0)


def ycf(data, k):
    ty = 0.0
    t0 = np.nanmean(data)
    count = 0
    for i in range(len(data) - k):
        for j in range(len(data[i]) - k):
            # nb: this is very rough!
            # Should really average _all_ points distance k apart (pythagorus...)
            # With enough data, this is OK (but may miss some features)
            t1 = data[i, j]
            t2 = data[i, j + k]
            t3 = data[i + k, j]
            if np.isnan(t1) or np.isnan(t2) or np.isnan(t3):
                continue
            ty += 0.5 * ((t1 - t0) * (t2 - t0) + (t1 - t0) * (t3 - t0))
            count += 1
    return ty / count if count > 0 else 0


y0 = ycf(tassie, 0)
print(y0)
yy = [ycf(tassie, k) / y0 for k in range(85)]

plt.plot(yy)
plt.show()

### 'anomaly' (just average _difference_ as function of distance)

In [ ]:
def dy(data, k):
    ty = 0.0
    count = 0
    for i in range(len(data) - k):
        for j in range(len(data[i]) - k):
            # nb: this is very rough!
            # Should really average _all_ points distance k apart (pythagorus...)
            # With enough data, this is OK (but may miss some features)
            t1 = data[i, j]
            t2 = data[i, j + k]
            t3 = data[i + k, j]
            if np.isnan(t1) or np.isnan(t2) or np.isnan(t3):
                continue
            ty += 0.5 * (np.abs((t1-t2)) + np.abs((t1-t3)))
            count += 1
    return ty / count if count > 0 else 0

yy = [dy(tassie, k) for k in range(85)]

plt.plot(yy)
plt.ylabel("$\\Delta T(d)$")
plt.xlabel("distance (grid points)")
plt.show()

### "Perfectly" clumpy example:

In [ ]:
def f(x,y):
    return np.cos(x/2)*np.cos(y/2)

x = np.arange(-25, 25, 1)
y = np.arange(-25, 25, 1)

X, Y = np.meshgrid(x, y)
Z = f(X,Y)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)

In [ ]:
y0 = ycf(Z, 0)
print(y0)
yy = [ycf(Z, k) / y0 for k in range(40)]

plt.plot(yy, "x-")
plt.title("Spatial correlation")
plt.xlabel("Distance")
plt.show()

In [ ]:
yy = [dy(Z, k) for k in range(40)]

plt.plot(yy, "x-")
plt.title("Difference")
plt.xlabel("Distance")
plt.show()